# README.md
#get baseline T and PSA dates and values for the salvage cohort

## Table of Contents
* [Read data, clean up and get stats](#first-bullet)
* [Get the baseline T dates and values](#second-bullet)
* [Get the T recovery status](#third-bullet)

In [1]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
import itertools

from matplotlib import pyplot
from pylab import *
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns

# Read data, clean up and get stats <a class="anchor" id="first-bullet"></a>

In [2]:
df0=pd.read_excel("./data/down_from_270pts_manual_reviewed.xlsx", header=0)
list_pt=df0["person_id"].to_list()
df0.fillna('', inplace=True)
print(shape(df0))
print("")

#get rid of empty lines
df_sa=pd.read_excel("./data/salvage_cohort.xlsx", header=0) 
df_sa_lo=pd.read_excel("./data/salvage_and_localized_cohort.xlsx", header=0) 

list_pt_sa=df_sa["person_id"].to_list()
df_sa.fillna('', inplace=True)
print(shape(df_sa))
print("")

list_pt_sa_lo=df_sa_lo["person_id"].to_list()
df_sa_lo.fillna('', inplace=True)
print(shape(df_sa_lo))
print("")

(270, 3)

(99, 4)

(98, 4)



In [5]:
df_sa_lo[:3]
df_sa_lo.drop(['person_id.1'], axis=1)

,person_id,ADT_start (manal reviewed mar16),ADT_end manual reviewed mar16
0,81,2014-04-15,2016-06-09 00:00:00
1,1257,2014-10-02,2015-07-08 00:00:00
2,1508,2016-08-03,2017-04-16 00:00:00
3,2061,2018-04-04,2019-02-06 00:00:00
4,2063,2017-11-15,2018-03-20 00:00:00
...,...,...,...
93,178232,2019-04-24,2020-01-31 00:00:00
94,180346,2019-04-05,2019-09-18 00:00:00
95,180780,2019-10-21,2020-01-15 00:00:00
96,186766,2019-05-06,2020-02-03 00:00:00


In [7]:
df_sa_lo.columns=(['index','person_id', 'ADT_start','ADT_end'])

In [32]:
df0[:3]

,person_id,ADT_start,ADT_end
0,2,2005-02-14 00:00:00,2005-12-12 00:00:00
1,13,2010-06-17 00:00:00,2015-03-17 00:00:00
2,25,2013-01-08 00:00:00,2014-02-18 00:00:00


In [9]:

print(shape(df_sa_lo))
df_fix=df_sa_lo[df_sa_lo.ADT_start!=""][df_sa_lo.ADT_start!=" "]
print(shape(df_fix))
df_fixfix=df_fix[df_fix.ADT_end!=""][df_fix.ADT_end!=" "]
print(shape(df_fixfix))

list_pt_new=df_fixfix["person_id"].to_list()

(98, 4)
(98, 4)
(98, 4)


In [11]:
df_tvalue=pd.read_excel("./data/pca_tresults_20210119_BP_fixed.xlsx", header=0)
#df3=pd.read_csv("just_start_empty.csv", header=0)
df_tvalue.replace(r'', np.NaN)
df_tvalue[:3]

df_tvalue_grouped=df_tvalue.groupby(["person_id"]).size().reset_index(name='counts')
list_pt_ttest=df_tvalue_grouped["person_id"].to_list()
print("list of pt ids with t test results")
print(len(list_pt_ttest))

print("overlapping with t test")
overlap_list1=[x for x in list_pt_new if x in  list_pt_ttest]
print(len(overlap_list1))

list of pt ids with t test results
4490
overlapping with t test
98


In [12]:
def closest(lst, K): 
      return lst[min(range(len(lst)), key = lambda i: abs(lst[i] - K))] 

In [13]:
df0[:5]

,person_id,ADT_start,ADT_end
0,2,2005-02-14 00:00:00,2005-12-12 00:00:00
1,13,2010-06-17 00:00:00,2015-03-17 00:00:00
2,25,2013-01-08 00:00:00,2014-02-18 00:00:00
3,38,2015-11-18 00:00:00,2016-07-18 00:00:00
4,81,2014-04-15 00:00:00,2016-06-09 00:00:00


In [14]:
list_pt_new

[81,
 1257,
 1508,
 2061,
 2063,
 2200,
 2623,
 2652,
 2655,
 3030,
 3905,
 3993,
 4595,
 5561,
 5791,
 5870,
 5873,
 7268,
 7611,
 7612,
 8272,
 8717,
 9124,
 9975,
 10945,
 11859,
 11861,
 12187,
 12779,
 13274,
 13613,
 14217,
 14221,
 14242,
 14278,
 14287,
 14900,
 15149,
 15495,
 16175,
 16558,
 17341,
 17395,
 17559,
 17580,
 17662,
 17710,
 17759,
 18303,
 18450,
 18821,
 19068,
 21047,
 21341,
 21480,
 21613,
 21832,
 21932,
 21938,
 22100,
 22833,
 23979,
 24562,
 24582,
 25809,
 25886,
 26071,
 26735,
 26854,
 27180,
 27207,
 27656,
 27766,
 27945,
 27966,
 30554,
 30816,
 31297,
 31492,
 31839,
 32230,
 32276,
 33500,
 33586,
 34207,
 34433,
 34547,
 34574,
 35103,
 35343,
 36249,
 36508,
 167118,
 178232,
 180346,
 180780,
 186766,
 187363]

# Get the baseline T dates and values <a class="anchor" id="second-bullet"></a>

In [15]:
##get the baseline T dates and values
result_baseline_list=[]
df1=df_fixfix
df2=df_tvalue

for y in list_pt_new:
    #print(y)    
    df1_pt=df1[(df1["person_id"] == y)]
    df1_pt['ADT_start'] = pd.to_datetime(df1_pt['ADT_start'])

    if(len(df1_pt.ADT_start)==0):
        a='NaN'
        status='NaN'
        tdate='NaN'
        tvalue='NaN'
        N_t_dates='NaN'
        N_t_dates_beforeADT='NaN'
            
    #if(df1_pt.ADT_start.values.any()!=""):
    if(len(df1_pt.ADT_start)!=0):
        
        a=df1_pt['ADT_start'].to_list()[0]

        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones before ADT start date, including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x <=a]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_beforeADT=len(cleaned_aaaa)
    
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            #status means no baseline TT avail
            status="N"
        #print(tdate)
        
        if(len(start_sorted_bin)>0):
            #apply the closet function
            #tdate=closest(start_sorted_bin,a)   
            df3_pt=df2_pt[df2_pt.result_date <=a]
            
            if not df3_pt.empty:
                tdate=df3_pt['result_date'].to_list()[-1]
                tvalue=df3_pt['test_results'].to_list()[-1]
                #status means yes there are baseline TT avail
                status="Y"

    print(y, a, tdate, tvalue, status, N_t_dates, N_t_dates_beforeADT)
    result_per_pt=[y,a,tdate,tvalue, status, N_t_dates, N_t_dates_beforeADT]
    result_baseline_list.append(result_per_pt)
    #validate 

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


81 2014-04-15 00:00:00 2014-04-10 00:00:00 313.07 Y 12 1
1257 2014-10-02 00:00:00 2014-10-02 00:00:00 234.83 Y 14 1
1508 2016-08-03 00:00:00 2016-07-20 00:00:00 757.0 Y 9 1
2061 2018-04-04 00:00:00 2018-03-02 00:00:00 187.63 Y 7 4
2063 2017-11-15 00:00:00 2016-12-20 00:00:00 349.75 Y 11 1
2200 2014-06-16 00:00:00 2014-05-21 00:00:00 127.98 Y 4 2
2623 2017-12-12 00:00:00 2017-11-28 00:00:00 679.66 Y 15 9
2652 2019-02-22 00:00:00 2019-02-22 00:00:00 393.35 Y 6 3
2655 2017-06-08 00:00:00 2017-06-08 00:00:00 278.1 Y 15 2
3030 2015-03-12 00:00:00 2015-03-02 00:00:00 181.89 Y 9 2
3905 2014-08-01 00:00:00 NaN NaN N 7 0
3993 2017-12-01 00:00:00 2017-10-09 00:00:00 10.47 Y 8 1
4595 2012-09-01 00:00:00 NaN NaN N 11 0
5561 2018-04-13 00:00:00 2018-03-30 00:00:00 408.0 Y 4 1
5791 2019-03-06 00:00:00 2019-02-25 00:00:00 502.86 Y 7 2
5870 2017-08-11 00:00:00 2017-04-26 00:00:00 290.63 Y 14 1
5873 2010-08-05 00:00:00 2010-04-19 00:00:00 240.84 Y 5 1
7268 2018-06-25 00:00:00 2018-06-18 00:00:00 269.03

In [16]:
df_baseline=pd.DataFrame(result_baseline_list)
df_baseline.columns=(['person_id', 'inital_ADT_start_date', 'Tdate_baseline', 'Tvalue_baseline', 'BaselineStatus',"N_T_dates","N_T_dates_beofreADT"])
df_baseline[:5]

#baseline. ok list
df_has_baseline=df_baseline[df_baseline.BaselineStatus=="Y"]
list_pt_with_tbaseline=df_has_baseline["person_id"].to_list()

In [17]:
print(len(list_pt_with_tbaseline))

91


In [18]:
df_baseline.to_csv("./data/270pts_salvage_localized_tbaseline.csv", encoding='utf-8', index=False)

# T recovery status

In [19]:
result_status_list_rec=[]
df1=df_fixfix
df2=df_tvalue

for y in list_pt_with_tbaseline:
    
    df1_pt=df1[(df1["person_id"] == y)]
    df1_pt['ADT_end'] = pd.to_datetime(df1_pt['ADT_end'])
    
    if(len(df1_pt.ADT_end) == 0):        
        b='NaN'
        #gap='NaN'
        tdate='NaN'
        tvalue='NaN'
        status="NaN"
        N_t_dates='NaN'
        N_t_dates_afterADT='NaN'

    if(len(df1_pt.ADT_end)!= 0):        
        b=df1_pt['ADT_end'].to_list()[0]
        
        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones after ADT end date including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x >=b]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_afterADT=len(cleaned_aaaa)
   
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            status="NaN"

#get the 300

        if(len(start_sorted_bin)>0):
            df3_pt=df2_pt[df2_pt.test_results >=300][df2_pt.result_date >=b]
            df4_pt=df2_pt[df2_pt.test_results <300][df2_pt.result_date >=b]

            if not df3_pt.empty:
                status ="Y"
                tdate=df3_pt['result_date'].to_list()[0]
                tvalue=df3_pt['test_results'].to_list()[0]
            
            #if not df4_pt.empty:
            if df3_pt.empty:
                status ="N"
                tdate=df4_pt['result_date'].to_list()[-1]
                tvalue=df4_pt['test_results'].to_list()[-1]
    
    print(y, b, tdate, tvalue,status, N_t_dates,N_t_dates_afterADT)
    result_status_per_pt=[y,b,tdate, tvalue, status,N_t_dates,N_t_dates_afterADT]
    result_status_list_rec.append(result_status_per_pt)
#result_status_list_rec=[]

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


81 2016-06-09 00:00:00 2017-04-03 00:00:00 309.59 Y 12 10
1257 2015-07-08 00:00:00 2017-07-07 00:00:00 313.36 Y 14 13
1508 2017-04-16 00:00:00 2018-02-07 00:00:00 778.86 Y 9 5
2061 2019-02-06 00:00:00 2019-10-09 00:00:00 39.2 N 7 2
2063 2018-03-20 00:00:00 2018-08-16 00:00:00 352.32 Y 11 10
2200 2014-11-03 00:00:00 2014-12-03 00:00:00 26.17 N 4 1
2623 2019-10-03 00:00:00 2020-05-28 00:00:00 177.1 N 15 1
2652 2019-06-01 00:00:00 2019-12-16 00:00:00 379.41 Y 6 3
2655 2017-11-07 00:00:00 2020-05-07 00:00:00 169.48 N 15 9
3030 2015-06-05 00:00:00 2020-10-12 00:00:00 6.2 N 9 7
3993 2020-02-06 00:00:00 NaN NaN NaN 8 0
5561 2019-06-21 00:00:00 NaN NaN NaN 4 0
5791 2020-12-01 00:00:00 NaN NaN NaN 7 0
5870 2017-11-13 00:00:00 2020-06-25 00:00:00 19.27 N 14 11
5873 2010-11-08 00:00:00 2012-09-13 00:00:00 214.46 N 5 4
7268 2019-01-19 00:00:00 2019-08-07 00:00:00 726.27 Y 8 6
7611 2018-09-12 00:00:00 2020-07-13 00:00:00 8.01 N 34 18
7612 2016-01-20 00:00:00 2016-05-18 00:00:00 570.37 Y 37 34
8272 

In [20]:
df_result_status_rec=pd.DataFrame(result_status_list_rec)
df_result_status_rec.columns=(['person_id', 'initial_ADT_end_date', 'Tdate_rec_norec', 'Tvalue_rec_norec', 'Status',"N_T_dates","N_T_dates_afterADT"])
df_result_status_rec

,person_id,initial_ADT_end_date,Tdate_rec_norec,Tvalue_rec_norec,Status,N_T_dates,N_T_dates_afterADT
0,81,2016-06-09,2017-04-03,309.59,Y,12,10
1,1257,2015-07-08,2017-07-07,313.36,Y,14,13
2,1508,2017-04-16,2018-02-07,778.86,Y,9,5
3,2061,2019-02-06,2019-10-09,39.2,N,7,2
4,2063,2018-03-20,2018-08-16,352.32,Y,11,10
...,...,...,...,...,...,...,...
86,167118,2020-02-19,NaT,NaN,NaN,2,0
87,180346,2019-09-18,2019-10-09,14.55,N,3,2
88,180780,2020-01-15,2020-05-13,10.46,N,4,2
89,186766,2020-02-03,2020-09-02,730.6,Y,10,4


In [21]:
df_result_status_rec

,person_id,initial_ADT_end_date,Tdate_rec_norec,Tvalue_rec_norec,Status,N_T_dates,N_T_dates_afterADT
0,81,2016-06-09,2017-04-03,309.59,Y,12,10
1,1257,2015-07-08,2017-07-07,313.36,Y,14,13
2,1508,2017-04-16,2018-02-07,778.86,Y,9,5
3,2061,2019-02-06,2019-10-09,39.2,N,7,2
4,2063,2018-03-20,2018-08-16,352.32,Y,11,10
...,...,...,...,...,...,...,...
86,167118,2020-02-19,NaT,NaN,NaN,2,0
87,180346,2019-09-18,2019-10-09,14.55,N,3,2
88,180780,2020-01-15,2020-05-13,10.46,N,4,2
89,186766,2020-02-03,2020-09-02,730.6,Y,10,4


In [22]:
df_result_status_rec.to_csv("./data/270pt_salvage_recovery_status_Mar18.csv", encoding='utf-8', index=False)

In [24]:
df_rec_pt=df_result_status_rec[df_result_status_rec.Status=="Y"]
df_nonrec_pt=df_result_status_rec[df_result_status_rec.Status=="N"]

list_pt_rec=df_rec_pt["person_id"].to_list()
list_pt_nonrec=df_nonrec_pt["person_id"].to_list()

df_has_baseline=df_baseline[df_baseline.BaselineStatus=="Y"]
list_pt_with_tbaseline=df_has_baseline["person_id"].to_list()
#
print(len(list_pt_rec))
print(len(list_pt_nonrec))
print(len(list_pt_with_tbaseline))

49
38
91


In [26]:
# assign status to 270 pts; rec, nonrec, not_included
df_270list=pd.read_excel("./data/270_total_list.xlsx", header=0)
list_pt=df_270list["person_id"].to_list()

In [27]:
print(shape(list_pt))

(270,)


In [ ]:
#all lists
#49 list_pt_rec
#38 list_pt_nonrec
#91 list_pt_with_tbaseline
#98 list_pt_new 

# mark these lists in the 270 pt lists to merge with the rest 

In [37]:
result_assign_status_list=[]
#for the 270 pts, make decisions of 1) in the 229 list y/n, 2) baseline y/n, and 3) recov y/n
for y in list_pt:

    if y not in list_pt_new: 
        
        status_salvage_local="N"
        status_tbaseline="NaN"
        status_rec="NaN"
        
    if y in list_pt_new: 
        status_salvage_local="Y"
        status_tbaseline="NaN"
        status_rec="NaN"
        
        if y in list_pt_with_tbaseline: 
            status_tbaseline="Y"
            if y in list_pt_rec:
                status_rec="Y"                 
            if y in list_pt_nonrec:
                status_rec="N"  
            
        if y not in list_pt_with_tbaseline: 
            status_tbaseline="N"
            if y in list_pt_rec:
                status_rec="NaN"  
            if y in list_pt_nonrec:
                status_rec="NaN"    
                #print(status_rec)
                
    print(y,status_manual_ok,status_tbaseline,status_rec)
    result_status=[y,status_salvage_local,status_tbaseline,status_rec]
    result_assign_status_list.append(result_status)

2 N NaN NaN
13 N NaN NaN
25 N NaN NaN
38 N NaN NaN
81 N Y Y
110 N NaN NaN
119 N NaN NaN
780 N NaN NaN
870 N NaN NaN
905 N NaN NaN
1257 N Y Y
1503 N NaN NaN
1508 N Y Y
1583 N NaN NaN
1704 N NaN NaN
2061 N Y N
2063 N Y Y
2200 N Y N
2278 N NaN NaN
2605 N NaN NaN
2623 N Y N
2652 N Y Y
2655 N Y N
2660 N NaN NaN
3030 N Y N
3129 N NaN NaN
3131 N NaN NaN
3155 N NaN NaN
3579 N NaN NaN
3905 N N NaN
3993 N Y NaN
4056 N NaN NaN
4235 N NaN NaN
4257 N NaN NaN
4319 N NaN NaN
4595 N N NaN
4717 N NaN NaN
4793 N NaN NaN
4854 N NaN NaN
5021 N NaN NaN
5041 N NaN NaN
5043 N NaN NaN
5394 N NaN NaN
5561 N Y NaN
5652 N NaN NaN
5791 N Y NaN
5870 N Y N
5873 N Y N
5875 N NaN NaN
6078 N NaN NaN
6104 N NaN NaN
6268 N NaN NaN
6505 N NaN NaN
6794 N NaN NaN
6841 N NaN NaN
7236 N NaN NaN
7268 N Y Y
7611 N Y N
7612 N Y Y
7798 N NaN NaN
7870 N NaN NaN
8020 N NaN NaN
8194 N NaN NaN
8272 N Y N
8302 N NaN NaN
8544 N NaN NaN
8717 N N NaN
8758 N NaN NaN
9124 N N NaN
9232 N NaN NaN
9233 N NaN NaN
9474 N NaN NaN
9541 N NaN NaN

In [38]:
df_result_3status=pd.DataFrame(result_assign_status_list)
df_result_3status.columns=(['person_id', 'status_salvage_local','status_tbaseline','status_rec'])

In [39]:
df_result_3status
df_rec= df_result_3status[(df_result_3status["status_rec"] == "Y")][(df_result_3status["status_tbaseline"] == "Y")]
print(shape(df_rec))

(49, 4)


/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [41]:
df_result_3status.to_csv("./data/270pt_salvage_assgin_3status_Mar18_NEW.csv", encoding='utf-8', index=False)

In [109]:
#check numbers